In [ ]:
### Lanelet 2

In [ ]:
import lanelet2
import lanelet2.core as lncore
from osgeo import ogr
from shapely import wkt as swkt
import os

# helper functions

# ----- making shapes ------
def make_geom_lanelet(llet):
    geom = ogr.Geometry(ogr.wkbMultiLineString)
    
    lb = ogr.Geometry(ogr.wkbLineString)
    for p in llet.leftBound:
        lb.AddPoint(p.x, p.y)
    rb = ogr.Geometry(ogr.wkbLineString)
    for p in llet.rightBound:
        rb.AddPoint(p.x, p.y)

    geom.AddGeometry(lb)
    geom.AddGeometry(rb)
    return geom

def make_geom_lanelet_map(lmap):
    geom = ogr.Geometry(ogr.wkbGeometryCollection)
    for llet in lmap.laneletLayer:
        geom.AddGeometry(make_geom_lanelet(llet))
    return geom

def make_geom_lanelet_path(lpath):
    geom = ogr.Geometry(ogr.wkbGeometryCollection)
    for llet in lpath:
        geom.AddGeometry(make_geom_lanelet(llet))
    return geom

def make_geom_lanelet_path_restricted(lpath):
    geom = ogr.Geometry(ogr.wkbGeometryCollection)
    rempath = lpath.getRemainingLane(lpath[0])
    for i in range(0, len(rempath)):
        geom.AddGeometry(make_geom_lanelet(rempath[i]))
    return geom


In [ ]:
# 1. creating (programmatically) and saving a simple lane

In [ ]:
# simple init an empty map
lmap = lncore.LaneletMap()

# define a lane with a given left bound and right bound (and attribs)
lb = lncore.LineString3d(lncore.getId(),
                [lncore.Point3d(lncore.getId(), 4, 0, 0), lncore.Point3d(lncore.getId(), 4, 2, 0),
                 lncore.Point3d(lncore.getId(), 3, 3, 1), lncore.Point3d(lncore.getId(), 3, 4, 1)])
cb = lncore.LineString3d(lncore.getId(),
                [lncore.Point3d(lncore.getId(), 6, 0, 0), lncore.Point3d(lncore.getId(), 6, 2, 0),
                 lncore.Point3d(lncore.getId(), 5, 3, 1), lncore.Point3d(lncore.getId(), 5, 4, 1)])
rb = lncore.LineString3d(lncore.getId(),
                [lncore.Point3d(lncore.getId(), 8, 0, 0), lncore.Point3d(lncore.getId(), 8, 2, 0),
                 lncore.Point3d(lncore.getId(), 7, 3, 1), lncore.Point3d(lncore.getId(), 7, 4, 1)])
# first lane, normal
llet1 = lncore.Lanelet(lncore.getId(), lb, cb, lncore.AttributeMap({"type": "lanelet"}))
# its twin lane is inverted (opposite driving sense)
llet2 = lncore.Lanelet(lncore.getId(), cb, rb, lncore.AttributeMap({"type": "lanelet"})).invert()

lmap.add(llet1)
lmap.add(llet2)

# create a projector in London
projector = lanelet2.projection.UtmProjector(lanelet2.io.Origin(51.40, -0.04, 0.0))
ll_path = os.path.join(os.path.dirname(os.path.abspath('')), "data/llet_generated_map.osm")
lanelet2.io.write(ll_path, lmap, projector)

In [ ]:
# 2. examing the result

In [ ]:
# what's in a lanelet map? 
for s in dir(lmap):
    if "__" not in s:
        print s

In [ ]:
# what did I draw ? 
# plot it here 
geom = make_geom_lanelet_map(lmap)
swkt.loads(geom.ExportToWkt())

In [ ]:
# let's look at the the lanelet layer 
lanes = [lane for lane in lmap.laneletLayer]
print("--- example of a lanelet ---\n\n%s" % lanes[0])
lane0 = lanes[0]
for s in dir(lane0):
    if "__" not in s:
        print "    * %s" % s

In [ ]:
print("* lanelet left bound")
for p in llet1.leftBound:
    print("    %s " % p)
print("* lanelet left bound")
for p in llet1.rightBound:
    print("    %s " % p)

In [ ]:
# 3. Now, open that map in JOSM
#       create a map with lanes and change the xml file to add lanelets

In [ ]:
#    if you have dawn linestrings in JOSM, the xml will be full of <way> and <nd> items
#       now create items <relation> and tag them "lanelet"
#
#    For example, 
#      <relation id="1010" version="1">
#        <member type="way" ref="-106947" role="left" />
#        <member type="way" ref="-106982" role="right" />
#            <tag k="type" v="lanelet" />
#            <tag k="subtype" v="road" />
#            <tag k="location" v="urban" />
#            <tag k="participant" v="vehicle:car" />
#       </relation>

In [ ]:
# let's read the tweaked_josm_map.osm

osm_path = os.path.join(os.path.dirname(os.path.abspath('')), "data/tweaked_map.osm")
print("using OSM: %s (exists? %s)" % (osm_path, os.path.exists(osm_path)))

projector = lanelet2.projection.UtmProjector(lanelet2.io.Origin(41.15,15.10))
lmap, err_list = lanelet2.io.loadRobust(osm_path, projector)

print("%d errors, %d lanes detected" % (len(err_list), len([l for l in lmap.laneletLayer])))

In [ ]:
# what did I load ? 
geom = make_geom_lanelet_map(lmap)
swkt.loads(geom.ExportToWkt())

In [ ]:
lanes = [l for l in lmap.laneletLayer]

lane = lanes[0]
print("\nlane: %s" % (lane))
bnd = lane.leftBound
for i in range(0, len(bnd)):
    print("    %d -> id: %d (x: %.2f, y: %.2f)" % (i, bnd[i].id, bnd[i].x, bnd[i].y))

lane = lanes[1]
print("\nlane: %s" % (lane))
bnd = lane.rightBound
for i in range(0, len(bnd)):
    print("    %d -> id: %d (x: %.2f, y: %.2f)" % (i, bnd[i].id, bnd[i].x, bnd[i].y))

In [ ]:
# bonus! the centerline

cline = lane.centerline
for i in range(0, 4):#len(cline)):
    print("    %d -> id: %d (x: %.2f, y: %.2f)" % (i, cline[i].id, cline[i].x, cline[i].y))
print("    ... etc ")

In [ ]:
# 4. reading an existing OSM map

In [ ]:
# let's download the example from lanelet git repo

osm_path = os.path.join(os.path.dirname(os.path.abspath('')), "data/mapping_example.osm")
print("using OSM: %s (exists? %s)" % (osm_path, os.path.exists(osm_path)))

projector = lanelet2.projection.UtmProjector(lanelet2.io.Origin(49.00,8.42))
lmap, err_list = lanelet2.io.loadRobust(osm_path, projector) 

print("%d errors, %d lanes detected" % (len(err_list), len([l for l in lmap.laneletLayer])))

In [ ]:
# what did I load ? 
geom = make_geom_lanelet_map(lmap)
swkt.loads(geom.ExportToWkt())

In [ ]:
nmax = 10
print("* example of the laneletLayer (first %d)\n" % nmax)
for ln in lmap.laneletLayer:
    print ln
    nmax -= 1
    if nmax == 0:
        break

In [ ]:
invlane = lmap.laneletLayer[8396043010843852718]
for a in invlane.attributes:
    print a

In [ ]:
lb = invlane.leftBound
print("left bound is a %s\n" % type(lb))
for a in lb.attributes:
    print a
print ("\noriginal")
for p in lb:
    print p
print("\nleft bound is inverted? %s" % lb.inverted())
print("\ninverted")
for p in lb.invert():
    print p

In [ ]:
# 5. Routing: go from a lanelet to another

In [ ]:
# create a graph of roads
# ... and traffic rules (Germany)

trafficRules = lanelet2.traffic_rules.create(lanelet2.traffic_rules.Locations.Germany, 
                                             lanelet2.traffic_rules.Participants.Vehicle)
graph = lanelet2.routing.RoutingGraph(lmap, trafficRules)

In [ ]:
# find all possible combination of routes that connect with a path

sp_map = dict()
for startLane in lmap.laneletLayer:
    for endLane in lmap.laneletLayer:
        if startLane.id != endLane.id:
            k = "%d_%d" % (startLane.id, endLane.id)
            rk = "%d_%d" % (endLane.id, startLane.id)
            sp = graph.shortestPath(startLane, endLane)
            if sp is not None:
                if k not in sp_map and rk not in sp_map:
                    sp_map[k] = sp
                
print("found %d connected lanes with shortest path" % len(sp_map))

In [ ]:
# print 5 of them

nfrom = 11369
nto = 11379
print("print the first %d" % nmax)
for k,v in sp_map.items():
    if nfrom == nto:
        break
    print("%d -> %s" % (nfrom, k))
    nfrom += 1

In [ ]:
# calculate the shortest path between two lanes

startLane = lmap.laneletLayer[1181845994370657488]
endLane = lmap.laneletLayer[7697222576222483732]

rt = graph.getRoute(startLane, endLane)
lpath = rt.shortestPath()

print ("found a path of %d lanes" % len(lpath))#rt.numLanes())

# what did I load ? 
geom = make_geom_lanelet_path(lpath)
swkt.loads(geom.ExportToWkt())

In [ ]:
# show the path 

for i in range(0, len(lpath)):
    geom =  make_geom_lanelet(lpath[i])
    if geom.GetGeometryCount() >= 2:
        print("%d -> %s..." % (1+i, geom.GetGeometryRef(0).ExportToWkt()[:60]))
    else:
        print("%d -> %s" % (1+i, geom.ExportToWkt()))

In [ ]:
# show the path (restricted to not change lane)
make_geom_lanelet_path_restricted(lpath)
swkt.loads(geom.ExportToWkt())

In [ ]:
# show the path (restricted to not change lane)
rempath = lpath.getRemainingLane(lpath[0])
for i in range(0, len(rempath)):
    geom =  make_geom_lanelet(rempath[i])
    if geom.GetGeometryCount() >= 2:
        print("%d -> %s" % (1+i, geom.GetGeometryRef(0).ExportToWkt()))
    else:
        print("%d -> %s" % (1+i, geom.ExportToWkt()))